In [26]:
cd /data/projects/windturbine-identification/MachineLearningCourse

/data/projects/windturbine-identification/MachineLearningCourse


In [34]:
"""
Created on Mon Feb 18 20:30:48 2019

@author: jschmidt
"""

import os, shutil, random

import pandas as pd
import sys
import gdal
import imp
import numpy as np


import scripts.windturbines.functions_pattern_recognition as fpr
imp.reload(fpr)
from scripts.windturbines.functions_pattern_recognition import get_param
from scripts.windturbines.functions_pattern_recognition import cop_predict
from scripts.windturbines.functions_pattern_recognition import check_image
from scripts.windturbines.functions_pattern_recognition import read_params

In [38]:
COUNTRY = "MIX"

train_dir = get_param(COUNTRY,"PATH_ML_IMAGES_TURBINES_TRAIN")
test_dir = get_param(COUNTRY,"PATH_ML_IMAGES_TURBINES_TEST")
validation_dir = get_param(COUNTRY,"PATH_ML_IMAGES_TURBINES_VALIDATION")

train_no_dir = get_param(COUNTRY,"PATH_ML_IMAGES_NOTURBINES_TRAIN")
test_no_dir = get_param(COUNTRY,"PATH_ML_IMAGES_NOTURBINES_TEST")
validation_no_dir = get_param(COUNTRY,"PATH_ML_IMAGES_NOTURBINES_VALIDATION")


#### delete directories if exist
#### create if not exist

shutil.rmtree(train_dir,ignore_errors=True)
shutil.rmtree(test_dir,ignore_errors=True)
shutil.rmtree(validation_dir,ignore_errors=True)
shutil.rmtree(train_no_dir,ignore_errors=True)
shutil.rmtree(test_no_dir,ignore_errors=True)
shutil.rmtree(validation_no_dir,ignore_errors=True)

os.makedirs(train_dir)
os.makedirs(test_dir)
os.makedirs(validation_dir)
os.makedirs(train_no_dir)
os.makedirs(test_no_dir)
os.makedirs(validation_no_dir)

src_dir_tb = get_param(COUNTRY,"PATH_RAW_IMAGES_TURBINES")
src_dir_notb = get_param(COUNTRY,"PATH_RAW_IMAGES_NOTURBINES")


#### copy turbine images
#### select subset depending on quality check file
quality_check = pd.read_csv(get_param(COUNTRY,"FILE_QUALITY_CHECK")).dropna(subset=["quality"])

quality_check_sub = quality_check.loc[quality_check['quality']>=90]

nmbfiles = quality_check_sub.shape[0]

quality_check_sub = quality_check_sub.reindex(np.random.permutation(quality_check_sub.index))


In [39]:
nmbfiles

6497

In [40]:
share_train=round(0.7*nmbfiles)
share_validate=round(0.85*nmbfiles)

print(nmbfiles)
print(share_train)
print(share_validate)

6497
4548
5522


In [41]:
cnt = 0

for i in range(0,nmbfiles-1):
    cnt += 1
        
    #file=str(int(quality_check_sub.iloc[i,:].loc["id"]))
    file = quality_check_sub.iloc[i,:].loc["id"]
    
    #print(file)
    src_ = os.path.join(src_dir_tb,file)
    if(cnt % 1000 == 0):
        print(cnt)
        print(src_)
    if(os.path.isfile(src_)):
        
        if(cnt<share_train):
            src = os.path.join(src_dir_tb,file)
            dst = os.path.join(train_dir,file)
            shutil.copyfile(src,dst)
        
        if(cnt>share_train and cnt<share_validate):
            src = os.path.join(src_dir_tb,file)
            dst = os.path.join(validation_dir,file)
            shutil.copyfile(src,dst)
        
        if(cnt>share_validate):
            src = os.path.join(src_dir_tb,file)
            dst = os.path.join(test_dir,file)
            shutil.copyfile(src,dst)
            
    #### copy no-turbine images

1000
data/aerialImages/GOOGLE/RESOLUTION19/MIX/raw/Turbines/7825_BR.png
2000
data/aerialImages/GOOGLE/RESOLUTION19/MIX/raw/Turbines/106.7026_37.8042_1.0_Huadian Ningxia Ningdong Phase 4_CN.png
3000
data/aerialImages/GOOGLE/RESOLUTION19/MIX/raw/Turbines/114_AT.png
4000
data/aerialImages/GOOGLE/RESOLUTION19/MIX/raw/Turbines/1047_AT.png
5000
data/aerialImages/GOOGLE/RESOLUTION19/MIX/raw/Turbines/7839_BR.png
6000
data/aerialImages/GOOGLE/RESOLUTION19/MIX/raw/Turbines/534_AT.png


In [42]:
cnt = 0

files = random.sample([x for x in os.listdir(src_dir_notb) if x.endswith(".png")], nmbfiles)
    

for i in range(0,nmbfiles-1):
    cnt += 1
    
    file = files[i]
    
    if(cnt % 1000 == 0):
        print(cnt)
        print(file)

    if(cnt<share_train):
        src = os.path.join(src_dir_notb, file)
        dst = os.path.join(train_no_dir, file)
        shutil.copyfile(src, dst)
            
    if(cnt>share_train and cnt<share_validate):
        src = os.path.join(src_dir_notb, file)
        dst = os.path.join(validation_no_dir, file)
        shutil.copyfile(src, dst)
            
    if(cnt>share_validate):
        src = os.path.join(src_dir_notb, file)
        dst = os.path.join(test_no_dir, file)
        shutil.copyfile(src, dst)
    



print('total training turbine images:', len(os.listdir(train_dir)))
print('total testing turbine images:', len(os.listdir(test_dir)))
print('total validation turbine images:', len(os.listdir(validation_dir)))
print('total training no-turbine images:', len(os.listdir(train_no_dir)))
print('total testing no-turbine images:', len(os.listdir(test_no_dir)))
print('total validation no-turbine images:', len(os.listdir(validation_no_dir)))

1000
5601_BR.png
2000
1316_BR.png
3000
112.1256_39.5896_0.9999284744262696_Guodian Shanxi Pinglubeishan Phase 1 Wind_CN.png
4000
22617_0.10267598181962967__DE.png
5000
3770_BR.png
6000
19136_0.05273553729057312__DE.png
total training turbine images: 4528
total testing turbine images: 972
total validation turbine images: 971
total training no-turbine images: 4547
total testing no-turbine images: 974
total validation no-turbine images: 973


16502